병렬 처리를 위한 효율 개선
- 특정 로직이 주어졌을 때, 두 개의 브랜치(노드 및 서브그래프)로 양분
- 기존 방식에는 하나의 분기로 움직이는 시퀀스
- 그러나, 해당 방식은 양분하는 병렬 처리 방식으로 시간 절약

<img src="/Users/seonghoe/인프런_랭그래프/lecture./langgraph/병렬처리_도식화.png">

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph

class AgentState(TypedDict):
    query: str
    answer: str # 세율
    # 우리에게 필요한 노드는 아래와 같음. (+ 이들에 대한 State 정의)
    tax_base_equation: str  # 과세표준 게산 수식
    tax_deduction: str  # 공제액 
    market_ratio: str   # 공정시장가액비율
    tax_base: str   # 과세표준 계산

graph_builder = StateGraph(AgentState)

환경변수 설정 및 STATE 정의 완료

과세표준 계산 수식 노드 생성 (→ 벡터 DB 내 존재)

In [9]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embedding_function = OpenAIEmbeddings(model='text-embedding-3-large')
vector_store = Chroma(
    embedding_function=embedding_function,
    collection_name='real_estate_tax',
    persist_directory='./real_estate_tax_collection'
)
retriever = vector_store.as_retriever(search_kwargs={'k': 3})

In [10]:
query = '5억짜리 집 1채, 10억짜리 집 1채, 20억짜리 집 1채를 가지고 있을 때 세금을 얼마나 내나요?'

In [13]:
from langchain_openai import ChatOpenAI
from langchain import hub

rag_prompt = hub.pull('rlm/rag-prompt')
llm = ChatOpenAI(model='gpt-4o')
small_llm = ChatOpenAI(model='gpt-4o-mini')

/opt/anaconda3/envs/inflearn_langgraph/lib/python3.12/site-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [21]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

# 리트리빙을 위한 체인
tax_base_retrieval_chain = (
    {'context': retriever, 'question': RunnablePassthrough()} 
    | rag_prompt 
    | llm 
    | StrOutputParser()
)

tax_base_equation_prompt = ChatPromptTemplate.from_messages([
    ('system', '사용자의 질문에서 과세표준을 계산하는 방법을 수식으로 나타내주세요. 부연설명 없이 수식만 리턴해주세요.'),
    ('human', '{tax_base_equation_information}')
])

# 결과를 수식으로 변환히기 위한 체인
tax_base_equation_chain = (
    {'tax_base_equation_information': RunnablePassthrough()}
    | tax_base_equation_prompt
    | llm
    | StrOutputParser()

)   

tax_base_chain = {'tax_base_equation_information': tax_base_retrieval_chain} | tax_base_equation_chain
 
def get_tax_base_equation(state: AgentState): 
    tax_base_equation_question = '주택에 대한 종합부동산세 게산시 과세표준을 계산하는 방법을 수식으로 표현해서 알려주세요'
    tax_base_equation = tax_base_chain.invoke(tax_base_equation_question)
    return {'tax_base_equation': tax_base_equation}

RunnableMap({"context": retriever, "question": RunnablePassthrough()})
- context = retriever.invoke(input)
- question = input
- RunnablePassthrough = '입력을 가공하지 않고 그대로 다음 단계로 넘기기 위한 장치'

In [ ]:
get_tax_base_equation({})
# {'tax_base_equation': '과세표준 = (주택의 공시가격 합계 - 공제금액) × 공정시장가액비율'}

{'tax_base_equation': '과세표준 = (주택의 공시가격 합계 - 공제금액) × 공정시장가액비율'}

공제액 노드 생성 

In [23]:
tax_deduction_chain = (
    {'context': retriever, 'question': RunnablePassthrough()} 
    | rag_prompt 
    | llm 
    | StrOutputParser()
)

def get_tax_deduction(state: AgentState): 
    tax_deduction_question = '주택에 대한 종합부동산세 계산시 공제금액을 알려주세요.'
    tax_deduction = tax_deduction_chain.invoke(tax_deduction_question)
    return {'tax_deduction':  tax_deduction}

In [ ]:
get_tax_deduction({})
# {'tax_deduction': '주택에 대한 종합부동산세 계산 시 공제금액은 1세대 1주택자의 경우 12억 원입니다. 법인 또는 법인으로 보는 단체의 경우 6억 원, 그 외의 경우는 9억 원이 공제됩니다.'}

{'tax_deduction': '주택에 대한 종합부동산세 계산 시 공제금액은 1세대 1주택자의 경우 12억 원입니다. 법인 또는 법인으로 보는 단체의 경우 6억 원, 그 외의 경우는 9억 원이 공제됩니다.'}

공정시장가액비율 노드 생성 (→웹 서치)

In [35]:
from langchain_community.tools import TavilySearchResults
from datetime import date

tavily_search_tool = TavilySearchResults(
    max_results=7,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=True
)

tax_market_ratio_prompt = ChatPromptTemplate.from_messages([
    ('system', f'아래 정보를 기반으로 사용자의 질문에 답변해주세요\n\nContext:\n{{context}}'),
    ('human', '{query}')
])

def get_market_ratio(state: AgentState):
    query = f'오늘 날짜:({date.today()})에 해당하는 주택 공시가격 공정시장가액비율은 몇 % 인가요?'
    context = tavily_search_tool.invoke(query)
    tax_market_ratio_chain = (
        tax_market_ratio_prompt
        | llm 
        | StrOutputParser()
    )
    market_ratio = tax_market_ratio_chain.invoke({'context': context, 'query': query})
    return {'market_ratio': market_ratio}

In [ ]:
get_market_ratio({})
# {'market_ratio': '2026년 기준으로 주택의 공정시장가액비율은 60%입니다.'}

{'market_ratio': '2026년 기준으로 주택의 공정시장가액비율은 60%입니다.'}

과세 표준 계산

In [58]:
from langchain_core.prompts import PromptTemplate

tax_base_calculation_prompt = PromptTemplate.from_template("""
주어진 내용을 기반으로 과세표준을 계산해주세요.
                                                                                            
과세표준 계산 공식: {tax_base_equation}
공제금액: {tax_deduction}
공정시장가액비율: {market_ratio}                                                                                                                             
사용자 주택 공시가격 정보: {query}                                
""")

def calculate_tax_base(state: AgentState):
    tax_base_equation = state['tax_base_equation']
    tax_deduction = state['tax_deduction']
    market_ratio = state['market_ratio']
    query = state['query']
    tax_base_calculation_chain = (
        tax_base_calculation_prompt
        | llm
        | StrOutputParser()
    )
    tax_base = tax_base_calculation_chain.invoke({
        'tax_base_equation': tax_base_equation,
        'tax_deduction': tax_deduction,
        'market_ratio': market_ratio,
        'query': query
    })

    print(f'tax_base == {tax_base}')
    return{'tax_base': tax_base}

In [59]:
initial_state = {
    'query': query,
    'tax_base_equation': '과세표준 = (주택의 공시가격 합계 - 공제금액) × 공정시장가액비율',
    'tax_deduction': '주택에 대한 종합부동산세 계산 시 공제금액은 1세대 1주택자의 경우 12억 원입니다. 법인 또는 법인으로 보는 단체의 경우 6억 원, 그 외의 경우는 9억 원이 공제됩니다.',
    'market_ratio': '2026년 기준으로 주택의 공정시장가액비율은 60%입니다.'
}

In [60]:
calculate_tax_base(initial_state)

tax_base == 주어진 정보를 기반으로 과세표준을 계산해 보겠습니다. 사용자에게는 세 채의 집이 있으며, 각각의 공시가격은 5억 원, 10억 원, 20억 원입니다.

1. **주택의 공시가격 합계 계산:**
   - 5억 원 + 10억 원 + 20억 원 = 35억 원

2. **공제금액 결정:**
   - 정보가 주어지지 않았으므로 일반적인 경우를 기준으로 계산합니다. 
   - 일반적인 경우(1세대 1주택자, 법인이 아닌 경우 등)에는 9억 원이 공제됩니다.

3. **공정시장가액비율:**
   - 2026년 기준으로 공정시장가액비율은 60%입니다.

4. **과세표준 계산:**
   - 과세표준 = (주택의 공시가격 합계 - 공제금액) × 공정시장가액비율
   - 과세표준 = (35억 원 - 9억 원) × 0.6
   - 과세표준 = 26억 원 × 0.6
   - 과세표준 = 15.6억 원

따라서, 과세표준은 15.6억 원입니다. 이 과세표준을 기준으로 종합부동산세가 계산되며, 세율은 해당 연도와 과세표준 구간에 따라 다르게 적용됩니다. 정확한 세금 계산을 위해서는 추가적인 세율 정보가 필요합니다.


{'tax_base': '주어진 정보를 기반으로 과세표준을 계산해 보겠습니다. 사용자에게는 세 채의 집이 있으며, 각각의 공시가격은 5억 원, 10억 원, 20억 원입니다.\n\n1. **주택의 공시가격 합계 계산:**\n   - 5억 원 + 10억 원 + 20억 원 = 35억 원\n\n2. **공제금액 결정:**\n   - 정보가 주어지지 않았으므로 일반적인 경우를 기준으로 계산합니다. \n   - 일반적인 경우(1세대 1주택자, 법인이 아닌 경우 등)에는 9억 원이 공제됩니다.\n\n3. **공정시장가액비율:**\n   - 2026년 기준으로 공정시장가액비율은 60%입니다.\n\n4. **과세표준 계산:**\n   - 과세표준 = (주택의 공시가격 합계 - 공제금액) × 공정시장가액비율\n   - 과세표준 = (35억 원 - 9억 원) × 0.6\n   - 과세표준 = 26억 원 × 0.6\n   - 과세표준 = 15.6억 원\n\n따라서, 과세표준은 15.6억 원입니다. 이 과세표준을 기준으로 종합부동산세가 계산되며, 세율은 해당 연도와 과세표준 구간에 따라 다르게 적용됩니다. 정확한 세금 계산을 위해서는 추가적인 세율 정보가 필요합니다.'}

세율 계산

In [ ]:
def calculate_tax_rate(state: AgentState)